## Quality indicators for Swiss acute hospitals – “établissement”

Each of these CSV files actually contains hundreds of tables with two headers. A missing column header for comments needs to be added by hand.

#### Indicateurs de qualité des hôpitaux suisses de soins aigus 2019 » selon l'établissement (CSV, de fr it)

477 tables. Sample headers:

* institution;indicator;taux observé 2014-2018;taux attendu 2014-2018;SMR 2014-2018;nombre de cas 2014-2018;taux observé 2019;taux attendu 2019;SMR 2019;nombre de cas 2019
* institution;indicator;durée de séjours (jours) 2014-2018;;;nombre de cas 2014-2018;durée de séjours (jours) 2019;;;nombre de cas 2019

#### Indicateurs de qualité des hôpitaux suisses de soins aigus 2019 » selon le site (CSV, de fr it)

720 tables. Sample headers:
* institution;indicator;;;;;taux observé 2019;taux attendu 2019;SMR 2019;nombre de cas 2019
* institution;indicator;;;;;durée de séjours (jours) 2019;;;nombre de cas 2019

[Téléchargement des fichiers CSV](https://www.bag.admin.ch/bag/fr/home/zahlen-und-statistiken/zahlen-fakten-zu-spitaelern/qualitaetsindikatoren-der-schweizer-akutspitaeler/qualitaetsindikatoren-dokumentation.html)

[Téléchargement des chiffres clés](https://www.bag.admin.ch/bag/fr/home/zahlen-und-statistiken/zahlen-fakten-zu-spitaelern/kennzahlen-der-schweizer-spitaeler.html)

[Documentation](https://www.bag.admin.ch/bag/fr/home/zahlen-und-statistiken/zahlen-fakten-zu-spitaelern/qualitaetsindikatoren-der-schweizer-akutspitaeler/qualitaetsindikatoren-wegleitung.html#anker2)

In [1]:
import pandas as pd
from math import nan
import re
from io import StringIO
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read each table separately

frames = []
counter = 0

with open('data/2019.csv', 'r', encoding='windows-1252') as f:
    current_frame = []
    for line in f.readlines():
        for search, replace in {'Hôpital Riviera­Chablais Vaud-Valais':'Hôpital Riviera-Chablais Vaud-Valais',
                                'See­Spital': 'See-Spital'}.items():
            line = line.replace(search, replace)
        if line[1:11] in ['nstitution', 'nstituzion']:
            if len(current_frame) > 0: # if the frame exists
                frames.append(current_frame)
            current_frame = [line.replace('\n', ';comment\n')]
            counter += 1
        else:
            current_frame.append(line)
counter

477

In [3]:
# Only keep tables with following headers:
# SMR 2014-2018;nombre de cas 2014-2018;SMR 2019;**nombre de cas 2019**
# SMR 2014-2018;Fallzahl 2014-2018;SMR 2019;**Fallzahl 2019**
# SMR 2014-2018;numero di casi 2014-2018;SMR 2019;**numero di casi 2019**

df_list = []
indicator_list = []
c = 0
for frame in frames:
    df = pd.read_csv(StringIO("\n".join(frame)), sep=';', on_bad_lines='warn')
    if 'SMR 2019' in df.columns:
        df_list.append(df)
        comments_count = len(df[df['comment'] == df['comment']])
        indicator = df.iloc[1][df.columns[1]]
        df.rename(columns={
            'Institution': 'institution',
            'instituzione': 'institution'
        }, inplace=True)
        institution_name = df['institution'].iloc[0]
        indicator_list.append(indicator)
        # print(f'Frame {c}: {institution_name}, {indicator}')
        # print(f'Frame {c}: indicator = {indicator}; {comments_count} comments found. Shape:', df.shape)
        c += 1
print(c)

159


In [4]:
# Read BAG list of hospitals
df_hosp = pd.read_excel('data/kzp19_data.xlsx', sheet_name='KZ2019_KZP19')

In [13]:
# Only keep categories 111 and 112 from the list
#hosp_names = df_hosp[df_hosp['Typ'].isin(['K111', 'K112'])]['Inst'].tolist()
hosp_names = df_hosp['Inst'].tolist()

In [14]:
df_central = [_df for _df in df_list if _df.iloc[0]['institution'] in hosp_names]

In [15]:
# We check all frames have the same columns (in 3 languages)
col_list = []
for _df in df_central:
    col_list.append("|".join(list(_df.columns)))
print(*set(col_list), sep='\n')

institution|indicator|taux observé 2014-2018|taux attendu 2014-2018|SMR 2014-2018|nombre de cas 2014-2018|taux observé 2019|taux attendu 2019|SMR 2019|nombre de cas 2019|comment
institution|Indikator|beobachtete Rate 2014-2018|erwartete Rate 2014-2018|SMR 2014-2018|Fallzahl 2014-2018|beobachtete Rate 2019|erwartete Rate 2019|SMR 2019|Fallzahl 2019|comment
institution|indicatore|tasso osservato 2014-2018|tasso atteso 2014-2018|SMR 2014-2018|numero di casi 2014-2018|tasso osservato 2019|tasso atteso 2019|SMR 2019|numero di casi 2019|comment


In [16]:
# Use French locale for columns
for _df in df_central:
    _df.columns = ['institution',
 'indicator',
 'taux observé 2014-2018',
 'taux attendu 2014-2018',
 'SMR 2014-2018',
 'nombre de cas 2014-2018',
 'taux observé 2019',
 'taux attendu 2019',
 'SMR 2019',
 'nombre de cas 2019',
 'comment']

## Export files for specific indicators

In [19]:
cesarienne_list = []
for _df in df_central:
    line = _df[_df['indicator'].str.startswith('G.1.4.P')]
    cesarienne_list.append(line.to_dict(orient='records')[0])
dfc = pd.DataFrame(cesarienne_list)
dfc.to_excel('cesarienne-2019.xlsx', index=False)

In [31]:
dfc[dfc['taux observé 2019'] != '*'].sort_values('taux observé 2019', ascending=False).to_excel('cesariennes_pourcent.xlsx', index=False)